<a href="https://colab.research.google.com/github/mihaela-manciu/Asigurarea-calitatii-in-aplicatii-bazate-pe-imagine/blob/main/ACABI_ASL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Directory where your images are stored (dataset folder)
base_dir = 'D:/TAID/Semestrul 2/ACABI/dataset'

# Image dimensions for VGG16
img_size = (224, 224)


In [ ]:

# Create data generators for training, validation, and testing
def create_data_generators(base_dir, img_size=(224, 224), batch_size=32):
    # ImageDataGenerator with augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Validation and Test generators (no augmentation, only rescaling)
    val_test_datagen = ImageDataGenerator(rescale=1. / 255)

    # Flow from directory (train, validation, test splits)
    train_generator = train_datagen.flow_from_directory(
        os.path.join(base_dir, 'train'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    val_generator = val_test_datagen.flow_from_directory(
        os.path.join(base_dir, 'val'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_generator = val_test_datagen.flow_from_directory(
        os.path.join(base_dir, 'test'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_generator, val_generator, test_generator



In [ ]:

# Build the VGG16 model with custom top layers
def build_vgg16_model(input_shape=(224, 224, 3), num_classes=26):
    # Load the pre-trained VGG16 model without the top layers (to add our own)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model to avoid retraining it
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top of the VGG16 base model
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)  # Add dropout for regularization
    x = Dense(num_classes, activation='softmax')(x)

    # Final model
    model = Model(inputs=base_model.input, outputs=x)

    return model


In [ ]:

# Compile the model
def compile_model(model):
    model.compile(optimizer=Adam(lr=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])



In [ ]:
# Train the model
def train_model(model, train_generator, val_generator, epochs=10, batch_size=32):
    # Define callbacks for saving the best model and early stopping
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
    early_stop = EarlyStopping(monitor='val_loss', patience=5)

    # Train the model
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=[checkpoint, early_stop]
    )

    return history



In [ ]:
# Evaluate the model on the test set
def evaluate_model(model, test_generator):
    test_loss, test_acc = model.evaluate(test_generator)
    print(f"Test Accuracy: {test_acc * 100:.2f}%")
    print(f"Test Loss: {test_loss:.4f}")



In [ ]:
def plot_training_history(history):
    # Plot accuracy
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


In [ ]:

# Main function to put everything together
def main():
    # Create data generators
    train_generator, val_generator, test_generator = create_data_generators(base_dir)

    # Build the VGG16 model
    model = build_vgg16_model(input_shape=(224, 224, 3), num_classes=26)  # ASL has 26 letters

    # Compile the model
    compile_model(model)

    # Train the model
    history = train_model(model, train_generator, val_generator, epochs=20, batch_size=32)

    # Plot the training history (accuracy and loss)
    plot_training_history(history)

    # Evaluate the model
    evaluate_model(model, test_generator)


# Run the main function
if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'D:/TAID/Semestrul 2/ACABI/dataset/train'